<a href="https://colab.research.google.com/github/F-Bafti/AI-Agents-and-Agentic-AI/blob/master/Modular_AI_Agent_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modular AI Agent
## G- Goal Implementation

In [8]:
!pip install langchain-community
!pip uninstall cohere langchain-cohere -y

!pip install cohere>=5.0.0
!pip install langchain-cohere

!pip install cohere==5.11.0 langchain-cohere==0.3.0

import json, os
from google.colab import userdata

api_key = userdata.get('COHERE_API_KEY')
os.environ['COHERE_API_KEY'] = api_key

# Define Prompt Class and Agent Response

In [2]:
import os
from dataclasses import dataclass, field
from typing import List, Dict, Any, Callable, Optional
import json
import time
import traceback
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


@dataclass
class Prompt:
    messages: List[Dict] = field(default_factory=list)
    tools: List[Dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)  # Fixing mutable default issue


def generate_response(prompt: Prompt) -> str:
    """Call LLM to get response"""

    # Initialize Cohere model with LangChain
    llm = ChatCohere(
        model="command-r-plus",  # or "command-r" for faster responses
        max_tokens=1024,
        temperature=0.3
    )

    messages = prompt.messages
    tools = prompt.tools

    result = None

    # Convert dict messages to LangChain message objects
    # For OpenAI to LangChain format
    langchain_messages = []
    for msg in messages:
        if msg["role"] == "system":
            langchain_messages.append(SystemMessage(content=msg["content"]))
        elif msg["role"] == "user":
            langchain_messages.append(HumanMessage(content=msg["content"]))
        elif msg["role"] == "assistant":
            langchain_messages.append(AIMessage(content=msg["content"]))

    # If there is no tools for LLM to use just get the text response
    if not tools:
        response = llm.invoke(langchain_messages)
        result = response.content
    else:
        # Part1: Handeling tools
        # Convert tools to the format expected by LangChain Cohere
        # Conversion from OpenAI to LangChain expected format for tool
        formatted_tools = []
        for tool in tools:
            if isinstance(tool, dict) and "function" in tool:
                # Extract the function definition for LangChain
                func_def = tool["function"]
                # Convert to Cohere-compatible format
                cohere_tool = {
                    "title": func_def["name"],
                    "description": func_def["description"],
                    "properties": func_def["parameters"].get("properties", {}),
                    "required": func_def["parameters"].get("required", [])
                }
                formatted_tools.append(cohere_tool)

        # Part2: Handeling tools
        # Bind tools to the model for function calling
        llm_with_tools = llm.bind_tools(formatted_tools)
        response = llm_with_tools.invoke(langchain_messages)

        # Part3: Handeling tools
        # If there is a tool, take the response of LLM and extract the tool
        # Otherwise just take the text from the LLM response
        if response.tool_calls:
            tool_call = response.tool_calls[0]
            result = {
                "tool": tool_call["name"],
                "args": tool_call["args"],
            }
            result = json.dumps(result)
        else:
            result = response.content

    return result


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Setting Up the GAME(Goal, Action, Memory, Environment)

In [3]:
# 1- Goal Class
# Creates a simple container to hold information about what the agent should accomplish
# priority: How important this goal is (lower numbers = higher priority)
# name: Short name for the goal (like "Gather Information")
# description: Detailed explanation of what to do
# frozen=True: Makes this immutable - once created, it can't be changed
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str



# 2- Action Class
# Creates a wrapper around a Python function to make it available to the AI
# name: What the AI will call this action (like "read_file")
# function: The actual Python function to execute
# description: Tells the AI what this function does
# parameters: Describes what arguments the function needs (JSON schema format)
# terminal: Whether calling this action should end the agent's execution
class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: Dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """Execute the action's function"""
        return self.function(**args)

# Creates a container to store all available actions
class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    # Looks up an action by its name
    def get_action(self, name: str) -> Action | None:
        return self.actions.get(name, None)

    # Returns ALL actions as a list
    def get_actions(self) -> List[Action]:
        """Get all registered actions"""
        return list(self.actions.values())



# 3- Memory Class
# Creates a container to store the conversation history
# self.items = []: An empty list to hold memory items
# Each item will be a dictionary representing one piece of the conversation
class Memory:
    def __init__(self):
        self.items = []  # Basic conversation history

    # Adds a new memory item to the end of the list
    def add_memory(self, memory: dict):
        """Add memory to working memory"""
        self.items.append(memory)

    # Returns the stored memories as a list
    def get_memories(self, limit: int = None) -> List[Dict]:
        """Get formatted conversation history for prompt"""
        return self.items[:limit]

    # Creates a new Memory object with system messages filtered out
    def copy_without_system_memories(self):
        """Return a copy of the memory without system memories"""
        filtered_items = [m for m in self.items if m["type"] != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory



# 4- Environment Class
# This is where actions actually get executed safely
# try: attempts to run the action
# action.execute(**args) calls the action with the provided arguments
# If it works: calls self.format_result() to package the result nicely
# If it fails: catches the error and returns error information instead of crashing
class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """Execute an action and return the result."""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        """Format the result with metadata."""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }


# Setting up the Agent Language and Function-Calling

In [4]:
# What this class do?
# Example: user_input = "Write a README for this project."
# What construct_prompt Does:
# The agent takes that simple user input and builds a complex prompt that includes:
# System instructions (the goals), Conversation history, Available tools, The user's request
# Prompt(
#     messages=[
#         {"role": "system", "content": "Goal: Read each file..."},
#         {"role": "user", "content": "Write a README for this project."},
#         {"role": "assistant", "content": '{"tool": "list_files", "args": {}}'},
#         {"role": "user", "content": "Tool result: [file1.py, file2.py]"}
#     ],
#     tools=[list_files_tool, read_file_tool, terminate_tool]
# )

class AgentLanguage:
    def __init__(self):
        pass

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:
        raise NotImplementedError("Subclasses must implement this method")

    def parse_response(self, response: str) -> dict:
        raise NotImplementedError("Subclasses must implement this method")



# This is a concrete implementation of the abstract AgentLanguage class
class AgentFunctionCallingActionLanguage(AgentLanguage):

    def __init__(self):
        super().__init__()

    # Takes a list of Goal objects and converts them into a system message
    # Creates a formatted string with separators to make it readable
    # Returns a list with one system message containing all goals
    def format_goals(self, goals: List[Goal]) -> List:
        # Map all goals to a single string that concatenates their description
        # and combine into a single message of type system
        sep = "\n-------------------\n"
        goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals])
        return [
            {"role": "system", "content": goal_instructions}
        ]


    def format_memory(self, memory: Memory) -> List:
        """Generate response from language model"""
        # Map all environment results to a role:user messages
        # Map all assistant messages to a role:assistant messages
        # Map all user messages to a role:user messages
        items = memory.get_memories()
        mapped_items = []
        for item in items:

            content = item.get("content", None)
            if not content:
                content = json.dumps(item, indent=4)

            if item["type"] == "assistant":
                mapped_items.append({"role": "assistant", "content": content})
            elif item["type"] == "environment":
                # Map environment results to user messages for Cohere compatibility
                mapped_items.append({"role": "user", "content": f"Tool result: {content}"})
            else:
                mapped_items.append({"role": "user", "content": content})

        return mapped_items

    def format_actions(self, actions: List[Action]) -> List:
        """Convert actions to LangChain-compatible tool format"""

        tools = []
        for action in actions:
            # Convert to OpenAI function format that LangChain can use
            tool_def = {
                "type": "function",
                "function": {
                    "name": action.name,
                    "description": action.description[:1024],
                    "parameters": action.parameters,
                }
            }
            tools.append(tool_def)

        return tools

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:

        prompt = []
        prompt += self.format_goals(goals)
        prompt += self.format_memory(memory)

        tools = self.format_actions(actions)

        return Prompt(messages=prompt, tools=tools)

    def adapt_prompt_after_parsing_error(self,
                                         prompt: Prompt,
                                         response: str,
                                         traceback: str,
                                         error: Any,
                                         retries_left: int) -> Prompt:

        return prompt

    def parse_response(self, response: str) -> dict:
        """Parse LLM response into structured format by extracting the ```json block"""

        try:
            return json.loads(response)

        except Exception as e:
            return {
                "tool": "terminate",
                "args": {"message": response}
            }



# The Agent Class

In [6]:
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "environment", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory

# Main Code

In [7]:

# Define the agent's goals
goals = [
    Goal(priority=1, name="Gather Information", description="Read each file in the project"),
    Goal(priority=1, name="Terminate", description="Call the terminate call when you have read all the files "
         "and provide the content of the README in the terminate message")
]

# Define the agent's language
agent_language = AgentFunctionCallingActionLanguage()

def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

def list_project_files() -> List[str]:
    try:
        # Get all files (not just .py files) to see what's available
        all_files = [f for f in os.listdir(".") if os.path.isfile(f)]
        py_files = [f for f in all_files if f.endswith(".py")]

        if not py_files:
            # If no .py files, return all text files
            text_files = [f for f in all_files if f.endswith(('.txt', '.md', '.json', '.yaml', '.yml', '.py', '.js', '.html', '.css'))]
            return sorted(text_files) if text_files else ["No readable files found in current directory"]

        return sorted(py_files)
    except Exception as e:
        return [f"Error listing files: {str(e)}"]

# Define the action registry and register some actions
action_registry = ActionRegistry()

action_registry.register(Action(
    name="list_project_files",
    function=list_project_files,
    description="Lists all readable files in the project directory (prioritizing Python files, but includes other text files if no Python files exist).",
    parameters={
        "type": "object",
        "properties": {},
        "required": []
    },
    terminal=False
))

action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type": "object",
        "properties": {
            "name": {"type": "string"}
        },
        "required": ["name"]
    },
    terminal=False
))

action_registry.register(Action(
    name="terminate",
    function=lambda message: f"{message}\nTerminating...",
    description="Terminates the session and prints the message to the user.",
    parameters={
        "type": "object",
        "properties": {
            "message": {"type": "string"}
        },
        "required": ["message"]
    },
    terminal=True
))

# Define the environment
environment = Environment()

# Create an agent instance
agent = Agent(goals, agent_language, action_registry, generate_response, environment)

# Run the agent with user input
if __name__ == "__main__":
    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)

    # Print the final memory
    print("\n" + "="*60)
    print("FINAL MEMORY:")
    print("="*60)
    for memory in final_memory.get_memories():
        print(memory)
        print("-" * 40)

Agent thinking...
Agent Decision: {"tool": "list_project_files", "args": {}}
Action Result: {'tool_executed': True, 'result': ['main.py'], 'timestamp': '2025-07-24T21:10:55+0000'}
Agent thinking...
Agent Decision: {"tool": "read_project_file", "args": {"name": "main.py"}}
Action Result: {'tool_executed': True, 'result': 'import os\nfrom dataclasses import dataclass, field\nfrom typing import List, Dict, Any, Callable, Optional\nimport json\nimport time\nimport traceback\nfrom langchain_cohere import ChatCohere\nfrom langchain_core.messages import HumanMessage, AIMessage, SystemMessage\n\n@dataclass\nclass Prompt:\n    messages: List[Dict] = field(default_factory=list)\n    tools: List[Dict] = field(default_factory=list)\n    metadata: dict = field(default_factory=dict)  # Fixing mutable default issue\n\n\ndef generate_response(prompt: Prompt) -> str:\n    """Call LLM to get response"""\n    \n    # Initialize Cohere model with LangChain\n    llm = ChatCohere(\n        model="command-r-